In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [52]:
df = pd.read_csv("")

In [53]:
x = df.content.values
x = [str (item) for item in x]
y = df.label.values
# encode labels
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=1 ,shuffle=True)

In [55]:
max_words = 100000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [56]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [57]:
model = RNN()
model.summary()
# Experiment with different optimisers
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 150, 50)           5000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_6 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [58]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
84/84 [==============================] - 11s 102ms/step - loss: 0.4754 - accuracy: 0.7671 - val_loss: 0.3740 - val_accuracy: 0.8294


In [59]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [64]:
accr = model.evaluate( test_sequences_matrix,Y_test)
print(accr)

179/179 [==============================] - 2s 10ms/step - loss: 0.3620 - accuracy: 0.8339
[0.36202359199523926, 0.8339179158210754]


In [71]:
from sklearn.metrics import classification_report
lstmPredict =model.predict(test_sequences_matrix)

In [75]:
def decodeLabel(score):
    return 1 if score>0.5 else 0

lstmPredict = [decodeLabel(score) for score in lstmPredict]

In [74]:
x = (classification_report(Y_test,lstmPredict))
print(x)

              precision    recall  f1-score   support

           0       0.81      0.94      0.87      3378
           1       0.88      0.68      0.77      2324

    accuracy                           0.83      5702
   macro avg       0.85      0.81      0.82      5702
weighted avg       0.84      0.83      0.83      5702

